In [1]:
from dask.distributed import Client, as_completed, wait, LocalCluster

In [2]:
cluster = LocalCluster()

2024-07-04 07:34:17,462 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 5 memory: 240 MB fds: 182>>
Traceback (most recent call last):
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/psutil/__init__.py", line 2166, in net_io_counters
    rawdict = _psplatform.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 12] Cannot allocate memory


In [3]:
client = cluster.get_client()
# client = Client(cluster)

In [4]:
cluster.scheduler_address, client.dashboard_link

('tcp://127.0.0.1:64129', 'http://127.0.0.1:8787/status')

In [5]:
executor = client.get_executor()

In [6]:
import numpy as np
import random

np.random.seed(42)

def my_randint(n):
    return np.random.randint(n)


rand_int, rand_int_2 = client.submit(np.random.randint, 10, 100, pure=False), client.submit(np.random.randint, 10, 100, pure=False)
rand_array = client.submit(np.random.sample, rand_int, pure=False)


In [26]:
futures = [client.submit(my_randint, 100) for i in range(10)]
results = [future.result() for future in futures]

results

[45, 45, 45, 45, 45, 45, 45, 45, 45, 45]

In [17]:
futures = [client.submit(my_randint, 100) for i in range(10)]

In [18]:
import concurrent.futures

concurrent.futures.wait(futures)

AttributeError: 'Future' object has no attribute '_condition'

In [13]:
list(executor.map(my_randint, 8*[100]))

[73, 73, 73, 73, 73, 73, 73, 73]

In [9]:
rand_int.result(), rand_int_2.result()

(60, 60)

In [13]:
rand_ints = client.submit(randint, 100, pure=False)
rand_ints.result()

87

2024-06-30 13:50:34,568 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 1 memory: 131 MB fds: 27>>
Traceback (most recent call last):
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/kuba/mambaforge/envs/europython-24/lib/python3.12/site-packages/psutil/__init__.py", line 2166, in net_io_counters
    rawdict = _psplatform.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 12] Cannot allocate memory


In [94]:
rand_array.result()

array([0.64994156, 0.15138395, 0.93491463, 0.85533582, 0.69231653,
       0.07504097, 0.88910608, 0.90056886, 0.43971331, 0.07422997,
       0.39572938, 0.2129719 , 0.60847031, 0.37033514, 0.35095807,
       0.32813383, 0.65029288, 0.12989228, 0.69951163, 0.91319887,
       0.94416824, 0.78963973, 0.86587826, 0.18312884, 0.95039526,
       0.13150257, 0.69773586, 0.79170537, 0.51943349, 0.21536584,
       0.96560799, 0.37804211, 0.17602276, 0.89392897, 0.32437725,
       0.25237542, 0.9067957 , 0.95818742, 0.48489151, 0.07272323,
       0.28967899, 0.57440831, 0.61805005, 0.83333353, 0.83581986,
       0.45176783, 0.68937492, 0.52150644, 0.7175384 , 0.52134997,
       0.67513669, 0.54031916, 0.41853442, 0.1825268 , 0.57257303,
       0.10525232])

In [33]:
from concurrent.futures import ProcessPoolExecutor

In [102]:
pp_executor = ProcessPoolExecutor(max_workers=2)

In [111]:
pp_rand_int = pp_executor.submit(random.randint, 10, 100)
pp_rand_int.result()

12

In [123]:
rng = np.random.default_rng(10)
rng.integers(10, 100)

79

In [124]:
np.random.randint(10, 100)

64

In [133]:
import uuid

random_uuid = pp_executor.submit(uuid.uuid4)

random_uuid.result()

UUID('3f09baa0-c31e-4ea3-b43c-2509fe1a5b0c')

In [31]:
import pandas as pd

def load_data() -> pd.DataFrame:
    return pd.DataFrame({"A": range(5)})

def a_mean(df: pd.DataFrame) -> float:
    return df["A"].mean()

In [32]:
a_mean(load_data())

2.0

In [41]:
df = client.submit(load_data)
df

<Future: finished, type: pandas.core.frame.DataFrame, key: load_data-d5df3d9fb551a91790a0b37ac294d2b8>

In [42]:
result = client.submit(a_mean, df)

In [36]:
result

<Future: finished, type: numpy.float64, key: a_mean-16cd6e68feda3ced429aaafa04b6ae2a>

In [37]:
result.result()

2.0

In [7]:
client.scatter

<bound method Client.scatter of <Client: 'tcp://127.0.0.1:64129' processes=5 threads=10, memory=32.00 GiB>>

In [11]:
data = np.random.sample((1000, 100))
data_id = client.scatter(data)

In [12]:
data_id

<Future: finished, type: numpy.ndarray, key: ndarray-f4e5e1a3c4d4d2753460533e53e79605>

In [13]:
data_id.__class__.__mro__

(distributed.client.Future, distributed.utils_comm.WrappedKey, object)

In [14]:
future = client.submit(np.mean, data_id)
future

<Future: pending, key: mean-c728dacb5336e3588a1015a286ef1583>

In [15]:
future.result()

0.5013301153148392

In [16]:
future.__class__.__mro__

(distributed.client.Future, distributed.utils_comm.WrappedKey, object)

In [29]:
import dask.array as da

persisted = client.persist(da.from_array(data))
persisted

dask.array<array, shape=(1000, 100), dtype=float64, chunksize=(1000, 100), chunktype=numpy.ndarray>

In [24]:
future = client.submit(np.mean, persisted)

In [28]:
result = future.result()
result.__class__.__mro__

(dask.array.core.Array, dask.base.DaskMethodsMixin, object)